Importing Libraries

In [1]:
import numpy as np
import pandas as pd

Importing dataset

In [3]:
dataset = pd.read_csv('../../data/data-6-model.csv')
X = dataset.iloc[:, 2:-3].values
y = dataset.iloc[:, -3].values

Encode Independant variables

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output = False), [0, 1])], remainder='passthrough')
X = ct.fit_transform(X)

Encode Dependant variables

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

Split dataset into Test set and Train set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Feature scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 16:] = sc.fit_transform(X_train[:, 16:])
X_test[:, 16:] = sc.transform(X_test[:, 16:])

Train SVM model on the Train set

In [9]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

Predict the Test set results

In [10]:
y_pred = classifier.predict(X_test)

Create Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[ 3  0  0  0  0]
 [ 1  9  0  1  0]
 [ 0  0  2  0  1]
 [ 0  0  0 10  3]
 [ 0  1  0  5  9]]
0.7333333333333333


Applying K-Fold Cross Validation

In [12]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy : {:.2f}".format(accuracies.mean()*100))
print("Standard Deviation : {:.2f}".format(accuracies.std()*100))

Accuracy : 72.16
Standard Deviation : 11.03


Applying GridSearch to find the best model and the best parameters

In [13]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1.0], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1.0], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy : {:.2f}".format(best_accuracy*100))
print("Best parameters : ", best_parameters)

Best Accuracy : 76.08
Best parameters :  {'C': 1.0, 'gamma': 0.8, 'kernel': 'rbf'}
